In [1]:
import os
import pandas as pd

In [23]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [3]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [4]:
df = pd.read_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]002_extract_actions.p"))

In [5]:
df.head()

,comment,dataset_file,model,note,page_id,page_ns,page_title,timestamp,user_id,user_ip,user_name,action_base,action_digit,action_language
0,/* wbsetdescription-add:1|ar */ مؤرخ ألماني,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2017-06-23 19:37:19+00:00,2437879,,Benseid Seid,wbsetdescription-add,1,ar
1,"/* wbsetlabel-add:1|sl */ Gerhard Raff, #quick...",wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2018-01-29 13:38:28+00:00,23475,,Sporti,wbsetlabel-add,1,sl
2,/* wbsetsitelink-add:1|enwiki */ Wolfram Wette,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-01-04 17:08:50+00:00,17848,,Assayer,wbsetsitelink-add,1,enwiki
3,/* wbsetreference-add:2| */ [[Property:P937]]:...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 08:21:58+00:00,38324,,Olaf Kosinsky,wbsetreference-add,2,
4,/* wbcreateclaim-create:1| */ [[Property:P1412...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 14:59:40+00:00,38324,,Olaf Kosinsky,wbcreateclaim-create,1,


In [35]:
label_dict = {}
mapping_file = os.path.join(cfg.get("directory", "dataset"), cfg.get("dataset", "wikidata_aux"), cfg.get("dataset", "wikidata_full_labels"))
if os.path.isfile(mapping_file): # load it manually if we have a mapping already
    df_labels_load = pd.read_csv(mapping_file)
    for i, r in df_labels_load.iterrows():
        label_dict[r['action_base']] = r['label']
# label_dict

In [7]:
action_counts = df['action_base'].value_counts()

In [36]:
for a,c in action_counts.iteritems():
    if a not in label_dict:
        print(a, c)
        label = input()
        label_dict[a] = label

In [37]:
df_label = pd.DataFrame({"action_base": list(label_dict.keys()), "label": list(label_dict.values())})

In [38]:
# store labels
df_label.to_csv(mapping_file, index=False)

In [40]:
for a,c in action_counts.iteritems():
    print(a, c)

wbcreateclaim-create 34365910
wbsetdescription-add 29508789
wbsetreference-add 12821614
wbsetclaim-create 12391814
wbsetlabel-add 11260870
wbsetdescription-set 8327748
wbeditentity-update 5824128
wbsetqualifier-add 4742348
wbsetsitelink-add 4561629
wbsetclaim-update 3116188
wbremoveclaims-remove 2941159
wbeditentity-create 2542185
wbsetlabel-set 1653396
wbsetaliases-add 1650484
clientsitelink-update 1230466
wbmergeitems-to 764768
wbmergeitems-from 724044
wbcreateredirect 600534
clientsitelink-remove 405785
wbsetsitelink-remove 309854
wbsetclaimvalue 257311
wbeditentity-create-item 253629
wbeditentity-override 201525
wbsetdescription-remove 183477
undo 176758
REVERT 128003
CREATE 97442
wbsetaliases-update 93481
wbsetlabel-remove 88956
wbsetaliases-remove 87856
wbsetsitelink-set 61283
wbsetaliases-set 54484
wbremovequalifiers-remove 40284
restore 40133
wbsetlabeldescriptionaliases 27211
wbsetqualifier-update 22429
add-form 20429
wbremovereferences-remove 19717
wbsetsitelink-set-both 1469

### Add label to revision

In [42]:
len(df)

141651850

In [ ]:
df = df.merge(right=df_label, how="left", on="action_base")

In [ ]:
df.head()

In [ ]:
label_counts = df['label'].value_counts()
label_counts